# Developing an AI application

앞으로 AI 알고리즘은 점점 더 많은 일상적인 애플리케이션에 통합될 것입니다. 예를 들어, 스마트폰 앱에 이미지 분류기를 포함하고 싶을 수도 있습니다. 이를 위해 수십만 개의 이미지로 학습된 딥러닝 모델을 전체 애플리케이션 아키텍처의 일부로 사용할 수 있습니다. 미래의 소프트웨어 개발에서 이러한 유형의 모델은 애플리케이션의 일반적인 구성 요소로 널리 사용될 것입니다. 

이 프로젝트에서는 다양한 꽃의 종을 인식할 수 있는 이미지 분류기를 학습하게 됩니다. 이를 활용하면 스마트폰 앱에서 카메라로 촬영한 꽃의 이름을 알려주는 기능을 구현할 수 있습니다. 실용적인 활용을 위해, 먼저 분류기를 학습한 후 이를 애플리케이션에서 사용할 수 있도록 내보낼 것입니다. 우리는 [이 데이터셋](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html)을 사용할 것이며, 102개의 꽃 종류가 포함되어 있습니다. 아래에서 몇 가지 예시를 확인할 수 있습니다.

<img src='assets/Flowers.png' width=500px>

이 프로젝트는 여러 단계로 나누어집니다:

* 이미지 데이터셋 로드 및 전처리
* 데이터셋을 사용한 이미지 분류기 학습
* 학습된 분류기를 사용하여 이미지 내용 예측

각 단계를 Python으로 구현할 수 있도록 안내할 것입니다.

이 프로젝트를 완료하면, 어떤 라벨이 지정된 이미지 세트에도 학습할 수 있는 애플리케이션을 갖게 됩니다. 여기서는 네트워크가 꽃을 학습하여 명령줄 애플리케이션으로 끝나겠지만, 새로운 기술을 어떻게 활용할지는 여러분의 상상력과 데이터셋 구축 노력에 달려 있습니다. 예를 들어, 자동차 사진을 찍으면 제조사와 모델을 식별하고 관련 정보를 찾아주는 앱을 만든다고 상상해 보세요. 직접 데이터셋을 만들어 새로운 것을 개발해 보세요.

먼저 필요한 패키지를 가져오는 것으로 시작하겠습니다. 코드의 시작 부분에 모든 import 문을 작성하는 것이 좋은 습관입니다. 노트북을 진행하면서 필요한 패키지가 있으면, 이곳에 import 문을 추가하세요.

In [9]:
# Imports here
# Basic Python libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# PyTorch imports
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

# Image processing
from PIL import Image

# Utility
import os
import json
from collections import OrderedDict

## 데이터 로드하기

여기서는 `torchvision`을 사용하여 데이터를 로드합니다 ([문서 참고](http://pytorch.org/docs/0.3.0/torchvision/index.html)). 데이터는 이 노트북과 함께 제공되어야 하지만, 그렇지 않은 경우 [여기에서 다운로드할 수 있습니다](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). 

데이터셋은 학습(training), 검증(validation), 테스트(testing) 세 부분으로 나누어집니다. 학습 데이터에는 랜덤 크기 조정, 자르기(cropping), 뒤집기(flipping) 등의 변환을 적용하는 것이 좋습니다. 이러한 변환을 통해 네트워크가 일반화(generalization) 능력을 갖추어 더 나은 성능을 발휘할 수 있습니다. 또한, 사전 학습된 네트워크의 요구 사항에 맞게 입력 데이터를 **224x224 픽셀**로 조정해야 합니다.

검증 및 테스트 세트는 네트워크가 한 번도 본 적 없는 데이터에 대한 성능을 평가하는 데 사용됩니다. 따라서 이 데이터에는 크기 조정(scaling)이나 회전(rotation)과 같은 변환을 적용하지 않는 것이 좋습니다. 대신, 적절한 크기로 이미지를 조정한 후 중앙을 자르는(cropping) 방식으로 전처리를 수행해야 합니다.

사용할 사전 학습된 네트워크는 ImageNet 데이터셋에서 학습되었습니다. 이 데이터셋에서는 각 색상 채널을 개별적으로 정규화(normalization)합니다. 따라서 세 가지 데이터셋(학습, 검증, 테스트)에 대해 네트워크가 기대하는 평균과 표준 편차를 맞춰야 합니다. 

- **평균(mean):** `[0.485, 0.456, 0.406]`
- **표준 편차(std):** `[0.229, 0.224, 0.225]`

이 값들은 ImageNet 데이터에서 계산된 것으로, 각 색상 채널을 0을 중심으로 하고 값의 범위를 -1에서 1 사이로 조정하는 역할을 합니다.

 

In [10]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [11]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                           [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                           [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                           [0.229, 0.224, 0.225])
    ])
}

# Load the datasets with ImageFolder
image_datasets = {
    'train': datasets.ImageFolder(train_dir, transform=data_transforms['train']),
    'valid': datasets.ImageFolder(valid_dir, transform=data_transforms['valid']),
    'test': datasets.ImageFolder(test_dir, transform=data_transforms['test'])
}

# Define the dataloaders
dataloaders = {
    'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=64, shuffle=True),
    'valid': torch.utils.data.DataLoader(image_datasets['valid'], batch_size=32),
    'test': torch.utils.data.DataLoader(image_datasets['test'], batch_size=32)
}

### 라벨 매핑

카테고리 라벨을 카테고리 이름으로 매핑하는 데이터도 로드해야 합니다. 이는 `cat_to_name.json` 파일에서 찾을 수 있습니다. 해당 파일은 [`json` 모듈](https://docs.python.org/2/library/json.html)을 사용하여 읽을 수 있는 JSON 객체입니다. 이를 통해 정수로 인코딩된 카테고리를 실제 꽃 이름과 매핑하는 딕셔너리를 얻을 수 있습니다.


In [12]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [ ]:
# TODO: Build and train your network
# 사전 학습된 VGG16 모델 불러오기
# VGG16 모델은 ImageNet 데이터셋에서 학습된 모델로, 이미지 분류 문제에 많이 사용되는 모델 중 하나입니다.
model = models.vgg16(pretrained=True)

# 모델의 모든 파라미터를 고정 (Gradient 업데이트 방지)
# 즉, 사전 학습된 가중치를 그대로 사용하고 새로운 가중치는 업데이트하지 않음
for param in model.parameters():
    param.requires_grad = False

# 새로운 분류기 정의
# OrderedDict를 사용하여 순차적인 레이어를 정의

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# TODO: Do validation on the test set

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [7]:
# TODO: Save the checkpoint 


/Users/duhwanjang/working/udacity_projects/python_ai/find_flower_ai/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/duhwanjang/working/udacity_projects/python_ai/find_flower_ai/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/5.. Train loss: 5.706.. Validation loss: 4.912.. Validation accuracy: 0.055


KeyboardInterrupt: 

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
model = models.vgg16(pretrained=True)

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes